In [3]:
import os
import opcua

In [17]:
client = opcua.Client("http://130.159.75.138:48050")

In [18]:
client.connect()

In [19]:
client.get_namespace_array()

['http://opcfoundation.org/UA/',
 'urn:BeckhoffAutomation:TcOpcUaGateway',
 'http://opcfoundation.org/UA/DI/',
 'http://unifiedautomation.com/Configuration/',
 'http://unifiedautomation.com/TagFile/']